# Notes & Code

## Chapter: Scaling with generators

Let's start with iterations, iteration is simply pass over each element in a sequence doing some operation on that. It's largelly used in for loops. But when a for loop is used, what happens under the hood? <br>
So actually python uses a built-in function called `iter()` transforming a sequence into a `iterator`, what points to next element each iteration. <br> 

- iterator: An iterator is like a moving pointer over the collections.

In [2]:
numbers = [1, 2, 3, 4, 5]

for num in numbers: print(num)

1
2
3
4
5


In [5]:
numbers = [1, 2, 3, 4, 5]

numbers_iter = iter(numbers)

for num in numbers_iter: print(num)
    
print(f'numbers type: {type(numbers)}')
print(f'numbers_iter type {type(numbers_iter)}')

1
2
3
4
5
numbers type: <class 'list'>
numbers_iter type <class 'list_iterator'>


So I have this situation for example in a for loop what python actually does is get some sequence and transform it into a iterator. And how python do that?<br>
Simple!<br>

There is a built-in function called iter() that get a sequence and return a iterator like showed previously. And how does iter() get the iterator?<br>

Many ways to do that but one specific is define the magic dunder method `__iter__`. In this point we have to define the difference between iterator and iterable.<br>

- iterable: Are objects that return an iterator when passed through iter() function.
- iterator: Are the returns of iter() function, an object that have give one element by time.

In [1]:
names = ['Tom', 'Shelly',' Garth']
names_it = iter(names)

In [5]:
next(names_it)

StopIteration: 

Analisando mais detalhadamente a operação iter() sobre um objeto iterável, podemos utilizar outra built-in function chamada next() para acessar o próximo elemento do iterador. aplicando next() sucessivas vezes, consigo acessar todos os elementos da sequencia, até que ao não ter mais elementos, ocorre um raise exception de um erro do tipo `StopIteration`.

In [57]:
names = ['Tom', 'Shelly',' Garth']
names_it = iter(names)

In [21]:
# neste caso, tenho também a presença de um valor default onde ao invés de um raise exception, terei obterei o valor default.
# este valor default continua ocorrendo indefinidamente ao acessar o iterator desta forma.

next(names_it, "Rodrigo")

'Rodrigo'

Vamos agora, começar a analisar onde esta definição de iteradores e iteráveis podem trazer ganho em sua utilização.

In [25]:
# Definição de uma função que calcula uma lista de quadrados de numeros, um a um recebendo como parametro o limite superior 
# da lista

def calculate_squares(__max):
    
    squares = list()
    
    for num in range(__max):
        
        squares.append(num ** 2)
        
    return squares

In [28]:
calculate_squares(4)

[0, 1, 4, 9]

Reparamos que neste caso passando para função o valor máximo de elementos que quero calcular o quadrado, é retornado uma lista de valores. Isto significa que preciso ocupar espaço na memória com uma lista de quadrados calculados. Ou seja, dependendo do numero de elementos este valor pode rapidamente explodir a memória. Imagine se quiser uma lista com 1 milhão de quadrados ou mais.<br>
Como comentado anteriormente, se uma classe definir o método mágico `__iter__` e o método `__next__` é possível obter um iterador passando este objeto pela função iter() e neste caso, também como visto anteriormente, não seria criada uma lista de valores e sim ocorreria a obtenção de um valor da sequencia por vez conforme necessário.<br>
Vamos ver a seguir como fazer esta definição utilizando orientação a objetos.

In [29]:
class SquareIterator():
    
    def __init__(self, max_value):
        
        self.max_value = max_value
        self.current_value = 0
        
    def __iter__(self):
        
        return self
    
    def __next__(self):
        
        if self.current_value >= self.max_value:
            
            raise StopIteration
            
        current_squared_value = self.current_value ** 2
        
        self.current_value += 1
        
        return current_squared_value

In [30]:
for num in SquareIterator(3):
    
    print(num)

0
1
4


In [31]:
for num in calculate_squares(3):
    
    print(num)

0
1
4


Utilizando tanto a classe como a função, quanto a classe eu consigo resolver o problema de iterar sobre uma lista de quadrados de números entre 0 e um valor máximo.
A diferença é que no caso da classe, eu não precisar gerar uma lista completa a cada chamada next, o objeto altera seu estado através do retorno de uma nova instância com seus atributos atualizados até atingir o limite.

Interessante, entramos agora em um novo questionamento...<br>
Ok, interessante essa criação da classe onde definindo os métodos `__iter__` e `__next__` consigo criar meu iterator sem precisar criar de fato uma sequencia inteira, acessando os elementos um a um conforme for preciso.<br>
A dúvida é, existe forma mais fácil e prática de fazer isso?<br>
E a resposta é sim!

### Generator Functions

As generate functions pelo que estou lendo tem muitas funcionalidades, mas a principal explorada neste texto é exatamente a praticidade na geração de iterators.<br>
Estas funções se parecem muito com uma função tradicional mas ao invés de ter um `return` ocorre uma nova palavra-chave, `yield`. E é exatamente este cara que faz toda a diferença.

In [1]:
def gen_nums():
    
    n = 0
    
    while n < 4:
        
        yield n
        
        n += 1

In [4]:
for num in gen_nums():
    
    print(num)

0
1
2
3


In [5]:
sequence = gen_nums()
type(sequence)

generator

Vamos as definições:

- `GENERATOR FUNCTION:` - É uma função na qual eu defino retorno através da palavra-chave yield e não da palavra `return`.

- `GENERATOR OBJECT:` - É o objeto retornado pela GENERATOR FUNCTION, e na real, ele é um iterator onde posso aplicar a built-in function next() e utilizar em for loops por exemplo.

In [10]:
next(sequence)

StopIteration: 

Aqui temos uma definição interessante sobre as GENERATOR FUNCTIONS e sua comparação com as regular functions.<br>
Enquanto funções regulares podem apresentar vários pontos de saída (return statement), elas apresentam apenas um ponto de entrada. Ou seja, toda vez que uma função regular é chamada, ela roda desde sua primeira linha.<br>
No caso das GENERATOR FUNCTIONS, cada yield statement funciona como ponto de saída e entrada. Ou seja, ao chamar a função uma primeira vez, ela roda até o primeiro yield, ao rodar a função novamente ela começa a rodar exatamente na linha seguinte ao yield retornado anteriormente. 

__Obs:__ Não é necessário dar um raise em uma excessão (StopIteration), no caso do GENERATOR OBJECT retornado por uma GENERATOR FUNCTION, ele jé é implementado nativamente.<br>

Vamos fazer alguns testes:

In [11]:
def gen_function():
    
    yield "Rodrigo Bernardo Medeiros"

In [33]:
sequence = gen_function()

In [35]:
next(sequence)

StopIteration: 

Seria uma prática interessande utilizar generator functions para definir fluxos de desenvolvimento, ou para operar parcialmente sobre valores?

In [36]:
def split_names(complete_name):
    
    names = complete_name.split()
    
    for name in names:
        
        yield name

In [42]:
names = [
    'rodrigo bernardo medeiros', 
    'adriane bernardo medeiros', 
    'roberval mauricio cesar'
]

first_name = list()
second_name = list()
third_name = list()

for name in names:
    
    separated_names = split_names(name)
    
    first_name.append(next(separated_names))
    second_name.append(next(separated_names))
    third_name.append(next(separated_names))

In [44]:
first_name

['rodrigo', 'adriane', 'roberval']

In [45]:
second_name

['bernardo', 'bernardo', 'mauricio']

In [46]:
third_name

['medeiros', 'medeiros', 'cesar']

Neste exemplo acima, eu utilizo uma GENERATOR FUNCTION para tratar determinado informação.

Vamos retornar ao exemplo do quadrado dos números, reformulando a solução utilizando uma GENERATOR FUNCTION.

In [47]:
def gen_square_numbers(max_value):
    
    current_value = 0
    
    while current_value < max_value:
        
        yield current_value ** 2
        
        current_value += 1

In [48]:
for num in gen_square_numbers(4):
    
    print(num)

0
1
4
9


Muito bom, neste caso foi possível reescrever a classe escrita anteriormente que definia diretamente um itarator por uma GENERATOR FUNCTION que retorna um GENERATOR OBJECT que no fundo também é um ITERATOR.
Repara-se pela verbosidade de cada código que utilizar GENERATOR FUNCTIONS parece ser muito mais natural, fácil de ler, manter e escalar.